# Formula 1 data analysis application season 2023
This is the 2023 version

This time, I start from what I did last year and try to have a better data visualisation

I will try to separate the battles

## Imports

In [2]:
import json

import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

## General Part
In this part, I will be interested on the general visualisation of the season
It will stay close to what I did last year

### Backend

In [12]:
# Import data
data_all = pd.read_json('data_2023.json')
data_all["index"] = data_all.groupby("pilot").cumcount()
data_all["index"] += 1

# Dataset of the cumulative points of each pilot
data_points = data_all[["pilot", "points"]].copy()
data_points["cum_points"] = data_points.groupby("pilot").cumsum()
data_points["index"] = data_points.groupby("pilot").cumcount()

# Extraction of pilots per team
with open("teams_2023.json") as jsonfile:
    teams_pilot = json.load(jsonfile)

data_team_points = {"team": [], "points": [], "cum_points": [], "index": []}
for i in data_points["index"].unique():
    for team in teams_pilot:
        data_team_points["team"].append(team)
        data_team_points["points"].append(data_points[data_points["index"] == i][data_points["pilot"].isin(teams_pilot[team])]["points"].sum())
        data_team_points["cum_points"].append(data_points[data_points["index"] == i][data_points["pilot"].isin(teams_pilot[team])]["cum_points"].sum())
        data_team_points["index"].append(i)
data_team_points = pd.DataFrame(data_team_points)

# last gp
last_gp = data_all.groupby("pilot").tail(1)

# colors
colors = json.loads(open("colors.json").read())

C:\Users\shayk\AppData\Local\Temp\ipykernel_12192\1167974563.py:19: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\shayk\AppData\Local\Temp\ipykernel_12192\1167974563.py:20: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\shayk\AppData\Local\Temp\ipykernel_12192\1167974563.py:19: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\shayk\AppData\Local\Temp\ipykernel_12192\1167974563.py:20: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\shayk\AppData\Local\Temp\ipykernel_12192\1167974563.py:19: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\shayk\AppData\Local\Temp\ipykernel_12192\1167974563.py:20: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\shayk\AppData\Local\Temp\ipykernel_12192\1167974563.py:19: UserWarning:

Boolean Series key will be reindexed to ma

### Frontend

#### Last GP

In [13]:
px.scatter(last_gp, x="pilot", y="best_lap", title="Time of fastest lap of each pilot (s)", template="plotly_dark")

In [14]:
px.scatter(last_gp, x="pilot", y="avg_speed", title="Mean speed of the fastes_lap of each pilot (kmh)", template="plotly_dark")

In [15]:
px.scatter(last_gp, x="pilot", y="time", title="Total time (s)", template="plotly_dark")
# no data displayed if the pilot didn't finish or had one lap of delay

In [18]:
fig = go.Figure(layout=go.Layout(title="Total time (s)", template="plotly_dark"))
fig.add_trace(go.Box(y=last_gp["best_lap"], name="boxplot fastest_lap time", boxpoints="all", jitter=0.1, pointpos=0, boxmean=True, text=last_gp["pilot"]))

In [17]:
fig = go.Figure(layout=go.Layout(title="Mean speed of the fastes_lap of each pilot (kmh)", template="plotly_dark"))
fig.add_trace(go.Box(y=last_gp["avg_speed"], name="boxplot fastest_lap time", boxpoints="all", jitter=0.1, pointpos=0, boxmean=True, text=last_gp["pilot"]))

In [16]:
fig = go.Figure(layout=go.Layout(title="Total time of race of each pilot (s)", template="plotly_dark"))
fig.add_trace(go.Box(y=last_gp["time"], name="boxplot total time", boxpoints="all", jitter=0.1, pointpos=0, boxmean=True, text=last_gp["pilot"]))

## Pilot battle Part

Here I am interested on the battles between pilots

### Backend

In [45]:
# Extract the battles between pilots
with open("battles.json") as jsonfile:
    battles = json.load(jsonfile)
pilot_battles = battles["battles"]["pilots"]

df_battles_pilot = []  # List of battles dataframes
for battle in pilot_battles:
    # Filter data with pilots of the battle
    pilots = list(battle.values())[0]
    df_bt = data_points[data_points["pilot"].isin(pilots)]
    
    # Compute the difference of points
    base_pilot = list(battle.keys())[0]
    df_bt["diff"] = df_bt["cum_points"] - df_bt[df_bt["pilot"] == base_pilot]["cum_points"].values[df_bt["index"]]
    df_battles_pilot.append(df_bt)

[   pilot  points  cum_points  index  diff
0    ham      15          15      0   -11
1    rus      12          12      0   -14
2    ver       0           0      0   -26
3    per       0           0      0   -26
4    lec      26          26      0     0
5    sai      18          18      0    -8
20   ham      16          31      1   -20
21   rus      18          30      1   -21
22   ver       2           2      1   -49
23   per       1           1      1   -50
24   lec      25          51      1     0
25   sai      16          34      1   -17,    pilot  points  cum_points  index  diff
8    pia       0           0      0    -6
9    nor       0           0      0    -6
10   oco       6           6      0     0
11   gas       2           2      0    -4
28   pia       0           0      1   -12
29   nor       0           0      1   -12
30   oco       6          12      1     0
31   gas       0           2      1   -10]


### Frontend

In [ ]:
px.line(df_battles_pilot[0], x="circuit_name", y="diff", color="pilot", color_discrete_map=colors["pilot"], title="Top Pilot battle", markers=True, template="plotly_dark")

In [ ]:
px.line(df_battles_pilot[1], x="circuit_name", y="diff", color="pilot", color_discrete_map=colors["pilot"], title="Middle Pilot battle", markers=True, template="plotly_dark")

## Teams battle Part

Here I am interested on the battles between teams

### Backend

In [46]:
# Extract the battles between teams
with open("battles.json") as jsonfile:
    battles = json.load(jsonfile)
team_battles = battles["battles"]["teams"]

df_battles_team = []  # List of battles dataframes
for battle in team_battles:
    # Filter data with teams of the battle
    teams = list(battle.values())[0]
    df_bt = data_team_points[data_team_points["team"].isin(teams)]
    
    # Compute the difference of points
    base_team = list(battle.keys())[0]
    df_bt["diff"] = df_bt["cum_points"] - df_bt[df_bt["team"] == base_team]["cum_points"].values[df_bt["index"]]
    df_battles_team.append(df_bt)

[        team  points  cum_points  index  diff
0    redbull       0           0      0   -44
1   mercedes      27          27      0   -17
2    ferrari      44          44      0     0
10   redbull       3           3      1   -82
11  mercedes      34          61      1   -24
12   ferrari      41          85      1     0,        team  points  cum_points  index  diff
3   mclaren       0           0      0    -8
4    alpine       8           8      0     0
13  mclaren       0           0      1   -14
14   alpine       6          14      1     0,            team  points  cum_points  index  diff
6   astonmartin       0           0      0   -11
7      williams       8           8      0    -3
8     alfaromeo      11          11      0     0
9          haas       0           0      0   -11
16  astonmartin       0           0      1   -21
17     williams       8          16      1    -5
18    alfaromeo      10          21      1     0
19         haas       0           0      1   -21]


### Frontend

In [ ]:
px.line(df_battles_team[0], x="circuit_name", y="diff", color="team", color_discrete_map=colors["team"], title="Top Team battle", markers=True, template="plotly_dark")

In [ ]:
px.line(df_battles_team[2], x="circuit_name", y="diff", color="team", color_discrete_map=colors["team"], title="Bottom Team battle", markers=True, template="plotly_dark")

NameError: name 'px' is not defined

#### Battle Compare

In [ ]:
# back end
# initialize the barplots of the battles
pilot_battle_compare = {
    "pilot": list(pilot_battles[0].values())[0],
    "podium": [0 for i in range(len(list(pilot_battles[0].values())[0]))],
    "win": [0 for i in range(len(list(pilot_battles[0].values())[0]))],
    "pole": [0 for i in range(len(list(pilot_battles[0].values())[0]))]
}

team_battle_compare = {
    "team": list(team_battles[0].values())[0],
    "podium": [0 for i in range(len(list(team_battles[0].values())[0]))],
    "win": [0 for i in range(len(list(team_battles[0].values())[0]))],
    "pole": [0 for i in range(len(list(team_battles[0].values())[0]))]
}

# compute the number of podiums, wins and poles of each pilot
for j in range(len(pilot_battle_compare["pilot"])):
    pilot = pilot_battle_compare["pilot"][j]
    pilot_battle_compare["podium"][j] = data_all[data_all["pilot"] == pilot][data_all["result"] < 4].shape[0]
    pilot_battle_compare["win"][j] = data_all[data_all["pilot"] == pilot][data_all["result"] == 1].shape[0]
    pilot_battle_compare["pole"][j] = data_all[data_all["pilot"] == pilot][data_all["grid"] == 1].shape[0]
for j in range(len(team_battle_compare["team"])):
    team = team_battle_compare["team"][j]
    team_battle_compare["podium"][j] = data_all[data_all["pilot"].isin(teams_pilot[team])][data_all["result"] < 4].shape[0]
    team_battle_compare["win"][j] = data_all[data_all["pilot"].isin(teams_pilot[team])][data_all["result"] == 1].shape[0]
    team_battle_compare["pole"][j] = data_all[data_all["pilot"].isin(teams_pilot[team])][data_all["grid"] == 1].shape[0]
pilot_battle_compare = pd.DataFrame(pilot_battle_compare)
team_battle_compare = pd.DataFrame(team_battle_compare)
df_test = pd.DataFrame({
        "pilot": 3*list(pilot_battle_compare["pilot"]),
        "type": ["podium"]*len(pilot_battle_compare["pilot"])+["win"]*len(pilot_battle_compare["pilot"])+["pole"]*len(pilot_battle_compare["pilot"]),
        "value": list(pilot_battle_compare["podium"]) + list(pilot_battle_compare["win"]) + list(pilot_battle_compare["pole"])
        })

# front end
fig = make_subplots(rows=3, cols=2, subplot_titles=("Top Pilot podiums", "Top Team podiums", "Top Pilot wins", "Top Team wins", "Top Pilot poles", "Top Team poles"))

fig.add_trace(go.Bar(x=pilot_battle_compare["pilot"], y=pilot_battle_compare["podium"], name="Podiums"), row=1, col=1)
fig.add_trace(go.Bar(x=pilot_battle_compare["pilot"], y=pilot_battle_compare["win"], name="Wins"), row=2, col=1)
fig.add_trace(go.Bar(x=pilot_battle_compare["pilot"], y=pilot_battle_compare["pole"], name="Poles"), row=3, col=1)

fig.add_trace(go.Bar(x=team_battle_compare["team"], y=team_battle_compare["podium"], name="Podiums"), row=1, col=2)
fig.add_trace(go.Bar(x=team_battle_compare["team"], y=team_battle_compare["win"], name="Wins"), row=2, col=2)
fig.add_trace(go.Bar(x=team_battle_compare["team"], y=team_battle_compare["pole"], name="Poles"), row=3, col=2)

fig.update_layout(title="Top Pilot battle", template="plotly_dark")

fig.show()


In [ ]:
px.line_polar(df_test, r="value", theta="type", color="pilot", color_discrete_map=colors["pilot"], template="plotly_dark", line_close=True)

In [ ]:
# diverging bar chart between lec and ham
fig = go.Figure(layout=go.Layout(template="plotly_dark"))
fig.add_traces(go.Bar(y=df_test["type"][df_test["pilot"]=="ham"], x=df_test["value"][df_test["pilot"]=="ham"], name="ham", orientation="h"))
fig.add_traces(go.Bar(y=df_test["type"][df_test["pilot"]=="lec"], x=-df_test["value"][df_test["pilot"]=="lec"], name="lec", orientation="h"))
fig.update_layout(barmode="relative", legend_orientation="h", title="Lec vs Ham")
fig.show()

### Per pilot

In [ ]:
fig = go.Figure(layout=go.Layout(title="Pilot's results", template="plotly_dark"))
for pilot in data_all["pilot"].unique():
    fig.add_traces(go.Box(y=gp_df["result"][gp_df["pilot"]==pilot], name=pilot, fillcolor=colors["pilot"][pilot], boxpoints="all", jitter=0.1, pointpos=0, boxmean=True))
fig.show()

### Per circuit

In [ ]:
fig = go.Figure(layout=go.Layout(title="Circuit average speed on fastest laps", template="plotly_dark"))
fig.add_traces(go.Box(x=gp_df["circuit_name"], y=gp_df["avg_speed"], text=gp_df["pilot"], boxpoints="all", jitter=0.1, pointpos=0, boxmean=True))

### global

In [ ]:
# imshow du result avec le pilot en x et le circuit en y
px.imshow(gp_df.pivot("pilot", "circuit_name", "result"), title="Pilot's results", text_auto=True, template="plotly_dark")

In [ ]:
fig = px.line(gp_df, x="circuit_name", y="result", color="pilot", color_discrete_map=colors["pilot"], title="Pilot's evolution of results", markers=True, template="plotly_dark")
fig.update_layout(yaxis={'autorange': 'reversed'})

In [ ]:
fig = px.scatter(gp_df, x="grid", y="result", color="pilot", color_discrete_map=colors["pilot"], animation_frame="circuit_name", animation_group="pilot", title="Pilot's evolution of results", template="plotly_dark")
fig.show()

## Laps

### Per lap

In [ ]:
fig = go.Figure(layout=go.Layout(title="Lap time", template="plotly_dark"))
fig.add_traces(go.Box(x=lap_df["lap"], y=lap_df["time"], boxpoints="all", jitter=0.1, pointpos=0, boxmean=True))

In [ ]:
fig = go.Figure(layout=go.Layout(title="Lap time of each pilot", template="plotly_dark"))
for pilot in lap_df["pilot"].unique():
    fig.add_traces(go.Box(y=lap_df["time"][lap_df["pilot"]==pilot], name=pilot, fillcolor=colors["pilot"][pilot], boxpoints="all", jitter=0.1, pointpos=0, boxmean=True))
# fig.add_traces(go.Box(x=lap_df["pilot"], y=lap_df["time"], boxpoints="all", jitter=0.1, pointpos=0, boxmean=True))
fig.show()

In [ ]:
px.line(lap_df, x="lap", y="time", color="pilot", color_discrete_map=colors["pilot"], title="Lap time evolution along the race", markers=True, template="plotly_dark")

In [ ]:
px.imshow(lap_df.pivot("pilot", "lap", "time"), title="Lap time along the race", text_auto=True, template="plotly_dark")

In [ ]:
fig = px.line(lap_df, x="lap", y="position", color="pilot", color_discrete_map=colors["pilot"], title="Pilot's position along the race", markers=True, template="plotly_dark")
fig.update_layout(yaxis={'autorange': 'reversed'})

## Circuit data

In [ ]:
fig = go.Figure(layout=go.Layout(title="Circuit Length", template="plotly_dark"))
fig.add_trace(go.Box(y=circuit_df["length"], name="boxplot circuit length", boxpoints="all", jitter=0.1, pointpos=0, boxmean=True, text=circuit_df["name"]))

In [ ]:
fig = go.Figure(layout=go.Layout(title="Circuit Lap Number", template="plotly_dark"))
fig.add_traces(go.Box(y=circuit_df["lap nb"], name="boxplot number of lap", boxpoints="all", jitter=0.1, pointpos=0, boxmean=True, text=circuit_df["name"]))

In [ ]:
px.scatter_geo(circuit_df, lat="lat", lon="long", hover_name="name", title="Circuit Location", template="plotly_dark")

In [ ]:
fig = go.Figure(layout=go.Layout(title="Lap number vs length", template="plotly_dark"))
fig.add_traces(go.Scatter(x=circuit_df["lap nb"], y=circuit_df["length"], mode="markers", text=circuit_df["name"], name="circuit data"))
fig.add_traces(go.Line(x=[i for i in range(30, 80)], y=[300000/i for i in range(30, 80)], name="expected tendancy"))

d:\Users\shayk\anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


